<a href="https://colab.research.google.com/github/pa2e37/dap-2024/blob/main/les07/rep7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Tuple
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score


DROP_COLS = ["Name", "Ticket", "Cabin"]


def load_data(filename: str) -> pd.DataFrame:
    df = pd.read_csv(filename, index_col="PassengerId")
    print("Данные успешно загружены!\n")
    print(df.head(7), "\n")
    return df


def describe_data(df: pd.DataFrame) -> None:
    print("Информация о типах данных:")
    df.info()
    print("\nПроверка пропусков:")
    print(df.isnull().sum(), "\n")


def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(columns=DROP_COLS)
    print(f"Удалены признаки: {', '.join(DROP_COLS)}\n")
    return df


def encode_sex(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df, columns=["Sex"], drop_first=False)
    print("Закодирован признак 'Sex' (male/female)\n")
    return df


def encode_embarked(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df, columns=["Embarked"], prefix="Embarked")
    print("One-Hot кодирование порта посадки завершено.\n")
    return df


def remove_nulls(df: pd.DataFrame) -> pd.DataFrame:
    before = len(df)
    df = df.dropna()
    print(f"Удалено строк с пропусками: {before - len(df)}\n")
    return df


def normalize_fare(df: pd.DataFrame) -> pd.DataFrame:
    df["Fare_norm"] = StandardScaler().fit_transform(df[["Fare"]])
    print("Нормализован признак 'Fare' → Fare_norm\n")
    return df


def analyze_fare(df: pd.DataFrame) -> float:
    diff = df[df.Survived == 1]["Fare"].mean() - df[df.Survived == 0]["Fare"].mean()
    print(f"Разница средних тарифов (выжившие - погибшие): {diff:.2f}\n")
    return diff


def plot_fare_hist(df: pd.DataFrame) -> None:
    plt.figure(figsize=(8, 5))
    plt.hist(df[df.Survived == 0]["Fare"], bins=30, alpha=0.6, label="Погибшие")
    plt.hist(df[df.Survived == 1]["Fare"], bins=30, alpha=0.6, label="Выжившие")
    plt.title("Распределение стоимости билета")
    plt.xlabel("Fare")
    plt.ylabel("Количество пассажиров")
    plt.legend()
    plt.show()


def prepare_xy(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    X = df.drop(columns=["Survived"])
    y = df["Survived"]
    print(f"Сформированы X и y. Размеры: X={X.shape}, y={y.shape}\n")
    return X, y


def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42
    )
    print("Разделение данных выполнено (75/25)\n")
    return X_train, X_test, y_train, y_test


def train_model(X_train, y_train) -> LogisticRegression:
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    print("Модель обучена\n")
    return model


def evaluate_model(model, X_test, y_test) -> float:
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)

    print("Матрица ошибок:\n", cm, "\n")
    print(f"Точность (accuracy): {acc:.4f}\n")

    return acc


def main():
    print(f"Python: {sys.version}")
    print(f"Numpy: {np.version.version}")
    print(f"Pandas: {pd.__version__}")
    print(f"Matplotlib: {mpl.__version__}\n")

    df = load_data("data/titanic.csv")
    describe_data(df)

    # Pipeline
    df = (
        df.pipe(clean_data)
        .pipe(encode_sex)
        .pipe(encode_embarked)
        .pipe(remove_nulls)
        .pipe(normalize_fare)
    )

    analyze_fare(df)
    plot_fare_hist(df)

    X, y = prepare_xy(df)
    X_train, X_test, y_train, y_test = split_data(X, y)

    model = train_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)


if __name__ == "__main__":
    main()

1. Что такое задача классификации?

Классификация — это задача предсказания категориальной (дискретной) метки на основе набора признаков объекта.

Формально:
Пусть:

X = {x1, …, xn} — множество объектов

Y = {y1, …, yk} — множество классов

Обучающая выборка состоит из пар (xi, yi)

Требуется построить отображение f: X -> Y, позволяющее предсказывать класс новых объектов.

Пример: определение того, является ли письмо спамом или нет.

2. Основные виды классификации

Бинарная — два класса.

Мультиклассовая — более двух классов; каждому объекту назначается ровно один.

Многометочная (multi-label) — объект может иметь несколько меток одновременно.

Многоклассовая с возможностью принадлежности нескольким классам — классы могут пересекаться.

3. Что характерно для несбалансированной классификации?

При несбалансированных данных один класс встречается значительно чаще другого.
Из-за этого:

модель может казаться точной, просто предсказывая доминирующий класс;

необходимы специальные методы обучения и метрики.

Как решать проблему:

использование весов классов;

oversampling малого класса (например, SMOTE);

уменьшение выборки большого класса;

оценка моделей не по accuracy, а по F1, Recall, Precision.

4. Особенности мультиклассовой классификации

Количество классов больше двух.

Каждый объект относится к одному единственному классу.

Часто требует разложения задачи на несколько бинарных.

Распространённые подходы:

One-vs-Rest (OvR)

One-vs-One (OvO)

5. Особенности бинарной классификации

Есть ровно два класса (например, 0 и 1).

Это наиболее простая форма классификации.

Удобно использовать пороговые предсказания.

Метрики:
Accuracy, Precision, Recall, F1-score, ROC-AUC.

6. Особенности многометочной классификации

Одному объекту может соответствовать несколько классов.

Каждая метка предсказывается отдельно.

Пример: текст может одновременно относиться к темам «спорт» и «экономика».

Методы:

Binary Relevance

Classifier Chains

Нейросети с несколькими выходами

7. Отличия стратегий "Один против всех" и "Один против одного"
One-vs-Rest (OvR)

Для K классов строится K моделей.

Каждая модель отделяет один класс от всех остальных.

Итоговое решение — класс с наибольшей уверенностью.

Плюсы: меньше моделей, быстро обучается.
Минусы: возможны проблемы с дисбалансом.

One-vs-One (OvO)

Строится K(K-1)/2 моделей: для каждой пары классов.

Итоговое решение выбирается голосованием.

Плюсы: лучше разделяет классы.
Минусы: много моделей → больше времени и ресурсов.

8. Что такое матрица ошибок?

Confusion Matrix — таблица, показывающая распределение правильных и ошибочных предсказаний модели.

Для бинарной классификации:

            Предсказано 0	    Предсказано 1
Истинно 0	      TN	               FP
Истинно 1	      FN	               TP

Где:

TP — верно предсказанные положительные

TN — верно предсказанные отрицательные

FP — ложноположительные

FN — ложноотрицательные

9. Как вычисляется Accuracy?

Accuracy показывает долю верных предсказаний.

Формула:

Accuracy = (TP + TN) / (TP + TN + FP + FN)

10. Принцип работы K-ближайших соседей

K-Nearest Neighbors (K-NN) — алгоритм, который классифицирует объект на основе близости к соседям из обучающей выборки.

Процесс:

Выбрать число соседей K.

Для нового объекта найти K ближайших точек.

Класс определяется большинством голосов среди этих соседей.

Гиперпараметры:

K

метрика расстояния

схема взвешивания соседей